1. Parameter Accuracy

This part will check how accurate the LLM parse the parameters for the function call after reading the query.

2. delivery rate

In [3]:
import json
#delivery rate
def getDeliveryRate(model, numPlan):
    delivered = 0
    with open (f'Output/{model}/plans/toolUsePlans.jsonl','r') as f:
         plans = [json.loads(line.strip()) for line in f]

    for record in plans:
        if record['plan'] != "":
            delivered += 1
    return delivered / numPlan

In [8]:
deliverayRate = getDeliveryRate('llama318b',300)
deliverayRate

0.8466666666666667

In [2]:
import json

In [4]:
#get the error list, rerun, wrong parameter, we don't have max steps. 

#test Llama's performance
with open ('Output/llama318b/plans/toolUseLogs.jsonl','r') as f:
    logs = [json.loads(line.strip()) for line in f]


In [47]:
logs[15]

{'index': 16,
 'log': [{'step': 1,
   'thought': 'Thought 1: I want to plan a 2-day trip with an expensive budget. I am interested in visiting history-oriented attractions. Please recommend some Irish restaurants with good freshness and a good environment. I would also like to stay in a hotel with a good location.',
   'action': 'Action 1: AccommodationSearch[Expensive Budget, [Good Location]]',
   'observation': "                                                       name                                                      address  latitude  longitude\n                                     AKA Rittenhouse Square                                                135 S 18th St 39.950365 -75.170453\n                                              Alexander Inn                                                301 S 12th St 39.946397 -75.160967\n                                 Aloft Philadelphia Airport                                              4301 Island Ave 39.891125 -75.234251\n          

In [42]:
validate_restaurant_parameters_format('Medium Budget, Middle Eastern, [Good Freshness]')

medium budget


ValueError: Wrong budget Input, valid ones include: cheap budget, moderate budget and expensive budget. Please try again.

In [32]:
for log in logs:
    for step in log['log']:
        if step['state'] != 'Successful':
            print(log['index'], step['step'],step['state'], step['action'])

11 3 Illegal args. Parameter Error Action 3: RestaurantSearch[Expensive Budget, US Cuisine, [Good Environment]]
11 4 Illegal args. Parameter Error Action 4: RestaurantSearch[Expensive Budget, US Cuisine, [Good Environment]]
11 5 same action 3 times repeated Action 5: RestaurantSearch[Expensive Budget, US Cuisine, [Good Environment]]
20 2 Illegal args. Parameter Error Action 2: RestaurantSearch[Good Value, Korean, [Good Flavor]]
20 3 Illegal args. Parameter Error Action 3: RestaurantSearch[Good Value, Korean, [Good Flavor]]
20 4 same action 3 times repeated Action 4: RestaurantSearch[Good Value, Korean, [Good Flavor]]
22 3 Illegal args. Parameter Error Action 3: RestaurantSearch[Expensive Budget, US Cuisine, [Good Environment, Good Flavor, Good Freshness]]
22 4 Illegal args. Parameter Error Action 4: RestaurantSearch[Expensive Budget, US Cuisine, [Good Environment, Good Flavor, Good Freshness]]
22 5 same action 3 times repeated Action 5: RestaurantSearch[Expensive Budget, US Cuisine, [G

In [45]:
logs[74]

{'index': 75,
 'log': [{'step': 1,
   'thought': 'Thought 1: I need to find the attractions that matches the preference.',
   'action': 'Action 1: AttractionSearch[Cheap budget,[Food Oriented]]',
   'observation': "                                    name                              address  latitude  longitude\n                     16th Street Seafood                       1542 Tasker St 39.930746 -75.171552\n                                Avril 50                       3406 Sansom St 39.953514 -75.192556\n                            Banh Mi Cali                          900 Arch St 39.953399 -75.154682\n                         Belmont Plateau              2000 Belmont Mansion Dr 39.990327 -75.212665\n                           Bloktoberfest                    17 Washington Ave 39.943856 -75.167177\n                                   Bui's                  38th St & Spruce St 39.951290 -75.199277\n             Campo's Philly Cheesesteaks                        214 Market St 39.9497

In [39]:
import re
def validate_restaurant_parameters_format(action_arg):
    pattern = r"(.*\s*.*),\s*(.*),\s*\[(.*)\]"
    match = re.match(pattern, action_arg)
    if not match:
        raise ValueError("Parameter format not match. Please try again. Valid Format: Budget, cuisine, preference list.")
    budget = match.group(1).lower()
    cuisine = match.group(2).lower()
    preference_list = match.group(3)

    budget_accepted = ['cheap budget', 'moderate budget','expensive budget']
    budgetInRange = False
    print(budget)
    if budget in budget_accepted:
        budgetInRange = True
    if not budgetInRange:
        raise ValueError("Wrong budget Input, valid ones include: cheap budget, moderate budget and expensive budget. Please try again.")

    cuisine_accepted = ["us","mexican","irish","french","italian","greek","indian","chinese","japanese","korean","vietnamese","thai","asian fusion","middle eastern"]
    print(cuisine)
    cuisineInRange = False
    if cuisine in cuisine_accepted:
        cuisineInRange = True
    if not cuisineInRange:
        raise ValueError("Cuisine not valid. Accepted cuisine is: US / Mexican / Irish / French / Italian / Greek / Indian / Chinese / Japanese / Korean / Vietnamese / Thai / Asian Fusion and Middle Eastern. Please try again.")

    preference_list = [p.lower().strip() for p in preference_list.split(',')]
    preference_core = [p.strip().split(' ')[-1] for p in preference_list]
    print(preference_core)

    preferenceInRange = True
    preferenceAccepted = ['',"flavor","freshness","service","environment","value"]
    for p in preference_core:
        if p not in preferenceAccepted:
            preferenceInRange = False

    if not preferenceInRange:
        raise ValueError("Wrong preference Input. Accepted inputs are: good flavor / good freshness / good healthy/ good service / good environment / good value. Please try again.")    
    return True